In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
import torch.utils.data as data_utils
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
import time

%matplotlib inline

In [ ]:
train=pd.read_csv('../data/train.csv', index_col='ID_code')
test=pd.read_csv('../data/test.csv', index_col='ID_code')
submission=pd.read_csv('../data/sample_submission.csv')

In [ ]:
x = train.loc[:,train.columns !='target']
print(x.shape)
x.head()

In [ ]:
y = train['target']
print(y.shape)
y.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
x=scaler.fit_transform(x)

In [ ]:
x.mean(), x.std()

In [ ]:
test = scaler.transform(test)

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
n_splits = 5
splits = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
splits=list(splits.split(x,y))

In [ ]:
print(len(splits))
splits[:3]

In [ ]:
train, valid = next(iter(splits))

In [ ]:
len(train), len(valid)

In [ ]:
from torch.optim.optimizer import Optimizer

class CyclicLR(object):

    def __init__(self, optimizer, base_lr=1e-3, max_lr=6e-3,
                 step_size=2000, mode='triangular', gamma=1.,
                 scale_fn=None, scale_mode='cycle', last_batch_iteration=-1):

        if not isinstance(optimizer, Optimizer):
            raise TypeError('{} is not an Optimizer'.format(
                type(optimizer).__name__))
        self.optimizer = optimizer

        if isinstance(base_lr, list) or isinstance(base_lr, tuple):
            if len(base_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} base_lr, got {}".format(
                    len(optimizer.param_groups), len(base_lr)))
            self.base_lrs = list(base_lr)
        else:
            self.base_lrs = [base_lr] * len(optimizer.param_groups)

        if isinstance(max_lr, list) or isinstance(max_lr, tuple):
            if len(max_lr) != len(optimizer.param_groups):
                raise ValueError("expected {} max_lr, got {}".format(
                    len(optimizer.param_groups), len(max_lr)))
            self.max_lrs = list(max_lr)
        else:
            self.max_lrs = [max_lr] * len(optimizer.param_groups)

        self.step_size = step_size

        if mode not in ['triangular', 'triangular2', 'exp_range'] \
                and scale_fn is None:
            raise ValueError('mode is invalid and scale_fn is None')

        self.mode = mode
        self.gamma = gamma

        if scale_fn is None:
            if self.mode == 'triangular':
                self.scale_fn = self._triangular_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = self._triangular2_scale_fn
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = self._exp_range_scale_fn
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode

        self.batch_step(last_batch_iteration + 1)
        self.last_batch_iteration = last_batch_iteration

    def batch_step(self, batch_iteration=None):
        if batch_iteration is None:
            batch_iteration = self.last_batch_iteration + 1
        self.last_batch_iteration = batch_iteration
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

    def _triangular_scale_fn(self, x):
        return 1.

    def _triangular2_scale_fn(self, x):
        return 1 / (2. ** (x - 1))

    def _exp_range_scale_fn(self, x):
        return self.gamma**(x)

    def get_lr(self):
        step_size = float(self.step_size)
        cycle = np.floor(1 + self.last_batch_iteration / (2 * step_size))
        x = np.abs(self.last_batch_iteration / step_size - 2 * cycle + 1)

        lrs = []
        param_lrs = zip(self.optimizer.param_groups, self.base_lrs, self.max_lrs)
        for param_group, base_lr, max_lr in param_lrs:
            base_height = (max_lr - base_lr) * np.maximum(0, (1 - x))
            if self.scale_mode == 'cycle':
                lr = base_lr + base_height * self.scale_fn(cycle)
            else:
                lr = base_lr + base_height * self.scale_fn(self.last_batch_iteration)
            lrs.append(lr)
        return lrs

In [ ]:
class Simple_NN(nn.Module):
    def __init__(self ,input_dim ,hidden_dim, dropout = 0.75):
        super(Simple_NN, self).__init__()
        
        self.inpt_dim = input_dim
        self.hidden_dim = hidden_dim
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(1, hidden_dim)
        self.fc2 = nn.Linear(int(hidden_dim*input_dim), 1)
        self.sigmoid = nn.Sigmoid()
        #self.fc3 = nn.Linear(int(hidden_dim/2*input_dim), int(hidden_dim/4))
        #self.fc4 = nn.Linear(int(hidden_dim/4*input_dim), int(hidden_dim/8))
        #self.fc5 = nn.Linear(int(hidden_dim/8*input_dim), 1)
        #self.bn1 = nn.BatchNorm1d(hidden_dim)
        #self.bn2 = nn.BatchNorm1d(int(hidden_dim/2))
        #self.bn3 = nn.BatchNorm1d(int(hidden_dim/4))
        #self.bn4 = nn.BatchNorm1d(int(hidden_dim/8))
    
    def forward(self, x):
        b_size = x.size(0) #256, batch_size 256
        x = x.view(-1, 1) #(256 X 200, 1)
        y = self.fc1(x) #(51200, 16)
        y = self.relu(y) #(51200, 16)
        y = y.view(b_size, -1)#(256, 51200 X 16 / 256)
        
        y= self.fc2(y) #(256, 1)
        out = self.sigmoid(y)
        
        return out

In [ ]:
model =Simple_NN(200,16)

In [ ]:
##Loss function
#loss_fn = FocalLoss(2)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001,weight_decay=1e-5) 
# Using Adam optimizer

In [ ]:
## 아래 Cycling learning rate 부터 재 설정 필요

In [ ]:
## Hyperparameter
n_epochs = 40
batch_size = 256

## Build tensor data for torch
train_preds = np.zeros((len(x)))
test_preds = np.zeros((len(test)))


valid_auc_scores =[]


## Start K-fold validation
for i, (train_idx, valid_idx) in enumerate(splits):  
    x_train = np.array(x)
    y_train = np.array(y)
    
    x_train_fold = torch.tensor(x_train[train_idx.astype(int)], dtype=torch.float)
    y_train_fold = torch.tensor(y_train[train_idx.astype(int), np.newaxis], dtype=torch.float32)
    
    x_val_fold = torch.tensor(x_train[valid_idx.astype(int)], dtype=torch.float)
    y_val_fold = torch.tensor(y_train[valid_idx.astype(int), np.newaxis], dtype=torch.float32)
    
    
    ######################Cycling learning rate########################
    step_size = 2000
    base_lr, max_lr = 0.001, 0.005  
    optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                                lr=max_lr)
    
    scheduler = CyclicLR(optimizer, 
                         base_lr=base_lr, 
                         max_lr=max_lr,
                         step_size=step_size, 
                         mode='exp_range',
                         gamma=0.99994
                )
    ###################################################################

    
    train = torch.utils.data.TensorDataset(x_train_fold, y_train_fold)
    valid = torch.utils.data.TensorDataset(x_val_fold, y_val_fold)
    
    
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)
    
    
    
    print(f'Fold {i + 1}')
    for e in range(n_epochs):
        start_time = time.time()
        
        avg_loss = 0.
        for i, (x_batch, y_batch) in enumerate(train_loader):
            y_pred = model(x_batch)
            ###################tuning learning rate###############
            if scheduler:
                #print('cycle_LR')
                scheduler.batch_step()

            ######################################################
            loss = loss_fn(y_pred, y_batch)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            avg_loss += loss.item()
    
    
        else:
            model.eval()
        
            avg_val_loss = 0.
            avg_val_auc = 0.
            for i, (x_batch, y_batch) in enumerate(valid_loader):
                y_pred = model(x_batch)
                loss = loss_fn(y_pred, y_batch)
            
                
                avg_val_loss += loss.item()
                avg_val_auc += round(roc_auc_score(y_batch.data.numpy().astype('int32'),y_pred.data.numpy()),4)
            
        
        valid_auc_scores.append(avg_val_auc /len(valid_loader))
        
        
        if e != 0 and max(valid_auc_scores) == (avg_val_auc /len(valid_loader)):
            torch.save(model, f'../model/model_nn_fold{i+1}_{avg_val_auc /len(valid_loader): .3f}_{e}.pt')
        
        
            model.train()
            
            elapsed_time = time.time() - start_time 
                
            print('Epoch {}/{} \t train_loss={:.4f} \t val_loss={:.4f} \t val_auc={:.4f} \t time={:.2f}s'.format(
                e + 1, n_epochs, avg_loss, avg_val_loss, avg_val_auc, elapsed_time))
     
    
    


## Inference

Now that the model is trained, we can use it for inference. We've done this before, but now we need to remember to set the model in inference mode with `model.eval()`. You'll also want to turn off autograd with the `torch.no_grad()` context.

In [ ]:
x_test = np.array(test)
x_test= torch.tensor(x_test, dtype=torch.float)

In [ ]:
print(f'model_path: ../model/model_nn{max(valid_auc_scores): .3f}_{valid_auc_scores.index(max(valid_auc_scores))}.pt')
model = torch.load(f'../model/model_nn{max(valid_auc_scores): .3f}_{valid_auc_scores.index(max(valid_auc_scores))}.pt')

In [ ]:
model.eval()
output = model(x_test)

In [ ]:
output=output.data.numpy()

In [ ]:
submission['target'] = output

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv(f'../submission/submission_nn{valid_auc_scores[e]: .3f}_190314.csv', index=False, header=True)